In [1]:
import optuna

In [2]:
!pip install xgboost

In [3]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv('Admission_Prediction.csv')

In [5]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [6]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [7]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [8]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [9]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [10]:
x=df.drop(columns=['Serial No.','Chance of Admit'])

In [11]:
y=df['Chance of Admit']
y


0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [12]:
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.000000,118.0,4.0,4.5,4.5,9.65,1
1,324.000000,107.0,4.0,4.0,4.5,8.87,1
2,316.558763,104.0,3.0,3.0,3.5,8.00,1
3,322.000000,110.0,3.0,3.5,2.5,8.67,1
4,314.000000,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.000000,108.0,5.0,4.5,4.0,9.02,1
496,337.000000,117.0,5.0,5.0,5.0,9.87,1
497,330.000000,120.0,5.0,4.5,5.0,9.56,1
498,312.000000,103.0,4.0,4.0,5.0,8.43,0


In [13]:
x.isnull().sum()

GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
dtype: int64

In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [15]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [16]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

{'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

{'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [17]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 10)
find_param.best_trial.params

[I 2022-12-09 00:37:10,745] A new study created in memory with name: no-name-919c4ef2-49f2-4a1b-bca8-a78b8ca841bb


[00:37:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0ac76685cf763591d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:0.23143
[1]	validation_0-rmse:0.22794
[2]	validation_0-rmse:0.22434
[3]	validation_0-rmse:0.22124
[4]	validation_0-rmse:0.21820
[5]	validation_0-rmse:0.21523
[6]	validation_0-rmse:0.21229
[7]	validation_0-rmse:0.20946
[8]	validation_0-rmse:0.20680
[9]	validation_0-rmse:0.20412
[10]	validation_0-rmse:0.20142
[11]	validation_0-rmse:0.19888
[12]	validation_0-rmse:0.19651
[13]	validation_0-rmse:0.19407
[14]	validation_0-rmse:0.19172
[15]	validation_0-rmse:0.18940
[16]	validation_0-rmse:0.18714
[17]	validation_0-rmse:0.18505
[18]	validation_0-rmse:0.18296
[19]	validation_0-rmse:0.18097
[20]	validation_0-rmse:0.17901
[21]	validation_0-rmse:0.17693
[22]	validation_0-rmse:0.17525
[23]	validation_0-rmse:0.17350
[24]	validation_0-rmse:0.17197
[25]	validation_0-rmse:0.17024
[26

C:\Users\AST_RI~1\AppData\Local\Temp/ipykernel_2596/1274079393.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
C:\Users\AST_RI~1\AppData\Local\Temp/ipykernel_2596/1274079393.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


[30]	validation_0-rmse:0.16282
[31]	validation_0-rmse:0.16161
[32]	validation_0-rmse:0.16037
[33]	validation_0-rmse:0.15893
[34]	validation_0-rmse:0.15779
[35]	validation_0-rmse:0.15650
[36]	validation_0-rmse:0.15542
[37]	validation_0-rmse:0.15442
[38]	validation_0-rmse:0.15341
[39]	validation_0-rmse:0.15247
[40]	validation_0-rmse:0.15151
[41]	validation_0-rmse:0.15072
[42]	validation_0-rmse:0.14999
[43]	validation_0-rmse:0.14916
[44]	validation_0-rmse:0.14834
[45]	validation_0-rmse:0.14761
[46]	validation_0-rmse:0.14682
[47]	validation_0-rmse:0.14624
[48]	validation_0-rmse:0.14558
[49]	validation_0-rmse:0.14499
[50]	validation_0-rmse:0.14441
[51]	validation_0-rmse:0.14374
[52]	validation_0-rmse:0.14317
[53]	validation_0-rmse:0.14266
[54]	validation_0-rmse:0.14215
[55]	validation_0-rmse:0.14162
[56]	validation_0-rmse:0.14111
[57]	validation_0-rmse:0.14070
[58]	validation_0-rmse:0.14025
[59]	validation_0-rmse:0.13985
[60]	validation_0-rmse:0.13942
[61]	validation_0-rmse:0.13903
[62]	val

[I 2022-12-09 00:37:11,273] Trial 0 finished with value: 0.017924321209886643 and parameters: {'lambda': 0.012343703813832131, 'alpha': 1.62332960703421, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.02, 'max_depth': 7, 'random_state': 2000, 'min_child_weight': 156}. Best is trial 0 with value: 0.017924321209886643.


[00:37:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0ac76685cf763591d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.

[0]	validation_0-rmse:1.57065
[1]	validation_0-rmse:9.36168
[2]	validation_0-rmse:65.40249
[3]	validation_0-rmse:454.61867
[4]	validation_0-rmse:3158.42172
[5]	validation_0-rmse:21960.69609
[6]	validation_0-rmse:153589.53698
[7]	validation_0-rmse:1073672.19514
[8]	validation_0-rmse:7506479.80118
[9]	validation_0-rmse:52479007.51183
[10]	validation_0-rmse:366896154.51408
[11]	validation_0-rmse:2564959697.27284
[12]	validation_0-rmse:17931944146.04706
[13]	validation_0-rmse:125363502789.89368
[14]	validation_0-rmse:876425197146.09595
[15]	validation_0-rmse:6127529533162.51953
[16]	validation_0-rmse:42838300934542.69531
[17]	validation_0-rmse:299483437503208.00000
[18]	validation_0-rmse:2093732368512206.00000
[19]	validation_0-rmse:14637743419875338.00000
[20]	validation_0-rmse:1023381298077

C:\Users\AST_RI~1\AppData\Local\Temp/ipykernel_2596/1274079393.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
C:\Users\AST_RI~1\AppData\Local\Temp/ipykernel_2596/1274079393.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
[W 2022-12-09 00:37:11,451] Trial 1 failed because of the following error: ValueError("could not convert string to float: '-nan(ind)'")
Traceback (most recent call last):
  File "C:\Users\Ast_Rishabh\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(

ValueError: could not convert string to float: '-nan(ind)'

In [19]:
xgb.XGBRFRegressor()

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               objective='reg:squarederror', predictor=None, random_state=None,
               reg_alpha=None, ...)

In [20]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.103952,6.051338,1.128802,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,309.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,316.558763,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,324.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [21]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.017924,2022-12-09 00:37:10.751155,2022-12-09 00:37:11.271191,0 days 00:00:00.520036,1.623330,0.8,0.012344,0.02,7,156,2000,0.4,COMPLETE
1,1,NaN,2022-12-09 00:37:11.275197,2022-12-09 00:37:11.449207,0 days 00:00:00.174010,4.145157,0.2,0.395535,8.00,10,46,2000,0.8,FAIL


In [22]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [23]:
import plotly


In [24]:
optuna.visualization.plot_optimization_history(find_param)

In [25]:
optuna.visualization.plot_slice(find_param)

In [30]:
best_param={'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [31]:
xgb_final_model=xgb.XGBRegressor(**best_param)

In [32]:
xgb_final_model.fit(x,y)

XGBRegressor(alpha=0.35848071012137483, base_score=0.5, booster='gbtree',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=0.5646022891371127,
             learning_rate=1, max_bin=256, max_cat_threshold=64,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=11, max_leaves=0,
             min_child_weight=30, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, ...)

In [33]:
xgb_final_model.score(x,y)

0.7818600803971398